In [1]:
import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objects as go
import pandas as pd
import numpy as np

from climada_petals.hazard import TCForecast
from climada.hazard import TCTracks

In [2]:
WIND_CONVERSION_FACTOR = 1. / 0.88

def filter_storm(fcast: TCForecast):
    """
    

    Parameters
    ----------
    fcast : climada.TCForecast
        The set of forecast data from ECMWF

    Returns
    -------
    fcast_filter : climada.TCForecast
        TCForecast class with storms which are named storm

    """
    tr_name_list = [tr.name for tr in fcast.data]
    tr_name_list = list(set(tr_name_list))
    
    fcast_filter = TCTracks()

    for tr_name in tr_name_list:

        if tr_name[:2].isdigit():
            continue
        
        else:
            fcast_filter.append(fcast.subset({
                'name': tr_name
            }).data)
    
    
    return fcast_filter

def _correct_max_sustained_wind_speed(tc_forecast: TCForecast,
                                      wind_conversion_factor: float = WIND_CONVERSION_FACTOR) -> None:
    """
    Converts the maximum sustained wind speed by a given factor. The sustained wind speed is defined as the wind speed
    which has not been underrun by any measurement within a given time interval.
    Note that the operation of multiplying by a constant factor is simplifying reality and a more complex method
    should be used eventually
    :param tc_forecast: The tracks object to which the modification is applied
    :param wind_conversion_factor: The factor by which the maximum sustained wind will be modified
    :return:
    """
    for dataset in tc_forecast.data:
        dataset['max_sustained_wind'] *= wind_conversion_factor

In [10]:
import matplotlib as mpl
import matplotlib.cm as cm_mp
from matplotlib.colors import BoundaryNorm, ListedColormap

# Function to categorize wind speed
def categorize_wind(speed):
    if speed < 17.49:
        return -1  # Tropical depression
    elif speed < 32.92:
        return 0  # Tropical storm
    elif speed < 42.7:
        return 1  # Category 1
    elif speed < 49.39:
        return 2  # Category 2
    elif speed < 58.13:
        return 3  # Category 3
    elif speed < 70.48:
        return 4  # Category 4
    elif speed < 1000:
        return 5  # Category 5
    else:
        return 999
    
SAFFIR_SIM_CAT = [17.49, 32.92, 42.7, 49.39, 58.13, 70.48, 1000]

CAT_COLORS = cm_mp.rainbow(np.linspace(0, 1, len(SAFFIR_SIM_CAT)))

CAT_NAMES = {
    -1: "Tropical Depression",
    0: "Tropical Storm",
    1: "Hurricane Cat. 1",
    2: "Hurricane Cat. 2",
    3: "Hurricane Cat. 3",
    4: "Hurricane Cat. 4",
    5: "Hurricane Cat. 5",
}

CUSTOM_LEGEND = ["Tropical Depression", "Tropical Storm", 
                 "Hurricane Cat. 1", "Hurricane Cat. 2", "Hurricane Cat. 3",
                 "Hurricane Cat. 4", "Hurricane Cat. 5"]

cmap = ListedColormap(colors=CAT_COLORS)
cmap_hex = []
for i in range(cmap.N):
    rgba = cmap(i)
    # rgb2hex accepts rgb or rgba
    cmap_hex.append(mpl.colors.rgb2hex(rgba))

In [4]:
BUFR_TRACKS_FOLDER = "./data/20240825000000"

In [5]:
tr_fcast = TCForecast()
tr_fcast.fetch_ecmwf(path=BUFR_TRACKS_FOLDER)
tr_filter = filter_storm(tr_fcast)
tr_filter.equal_timestep(3.)
_correct_max_sustained_wind_speed(tr_filter)

Processing:  10%|▉         | 5/51 [00:00<00:07,  6.15 files/s]

2024-10-31 15:57:17,225 - climada_petals.hazard.tc_tracks_forecast - WARNING - Longitude at time 0: only 1 variable value for 52 ensemble members, duplicating value to all members. This is only acceptable for lat and lon data at time 0.


Processing:  65%|██████▍   | 33/51 [00:03<00:01, 11.15 files/s]

2024-10-31 15:57:20,043 - climada_petals.hazard.tc_tracks_forecast - WARNING - Longitude at time 0: only 1 variable value for 52 ensemble members, duplicating value to all members. This is only acceptable for lat and lon data at time 0.


Processing: 100%|██████████| 51/51 [00:05<00:00,  8.98 files/s]


In [17]:
fig = go.Figure()

for track in tr_filter.data:
    df = pd.DataFrame({
        'lon': track['lon'],
        'lat': track['lat'],
        'wind_speed': track['max_sustained_wind'],
        'category': [categorize_wind(ws) for ws in track['max_sustained_wind']]
    })

    for i in range(len(df) - 1):
        fig.add_trace(go.Scattergeo(
            lon = df['lon'][i:i+2],
            lat = df['lat'][i:i+2],
            mode = 'lines',
            line = dict(width = 2, color = cmap_hex[df['category'][i]+1]),
            name = f"{track.name} - Cat {df['category'][i]}",
            showlegend = False
        ))

# Add invisible traces for legend
for category, color in zip(CUSTOM_LEGEND, cmap_hex):
    fig.add_trace(go.Scattergeo(
        lon = [None],
        lat = [None],
        mode = 'lines',
        line = dict(width = 2, color = color),
        name = category,
        showlegend = True
    ))

fig.update_layout(
    title = 'Tropical Cyclone Tracks',
    geo = dict(
        showland = True,
        showcountries = True,
        showocean = True,
        countrywidth = 0.5,
        landcolor = 'rgb(241, 232, 232)',
        oceancolor = 'rgb(255, 255, 255)',
        projection = dict(type = 'natural earth')
    ),
    legend_title = 'Saffir–Simpson Scale',
    legend = dict(
        yanchor = "top",
        y = 1,
        xanchor = "left",
        x = .95
    )
)

# Show the plot
fig.show()
fig.write_html("interactive_map_demo.html")